# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [203]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [204]:
# load data
dataFolder = r"C:\Users\jaker\Documents\RoboData-Experiment1"
data = "AllDataDFDelta20.csv"

df = pd.read_csv(os.path.join(dataFolder, data))

In [205]:
# fix index from read_csv
df = df.rename(columns={'Unnamed: 0': 'Seq'})
df = df.set_index('Seq')

In [206]:
df.head(10)

,OdomPosX,OdomPosY,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,TimeDelta1,TimeDelta11,TimeDelta21,TimeDelta31,TimeDelta41,TimeDelta51,TimeDelta61,TimeDelta71,TimeDelta81,TimeDelta91,TimeDelta101,TimeDelta111,TimeDelta121,TimeDelta131,TimeDelta141,TimeDelta151,TimeDelta161,TimeDelta171,TimeDelta181,TimeDelta191,TimeDelta201,OdomPosXDelta1,OdomPosXDelta11,OdomPosXDelta21,OdomPosXDelta31,OdomPosXDelta41,OdomPosXDelta51,OdomPosXDelta61,OdomPosXDelta71,OdomPosXDelta81,OdomPosXDelta91,OdomPosXDelta101,OdomPosXDelta111,OdomPosXDelta121,OdomPosXDelta131,OdomPosXDelta141,OdomPosXDelta151,OdomPosXDelta161,OdomPosXDelta171,OdomPosXDelta181,OdomPosXDelta191,OdomPosXDelta201,OdomPosYDelta1,OdomPosYDelta11,...,ImuAccelXDelta61,ImuAccelXDelta71,ImuAccelXDelta81,ImuAccelXDelta91,ImuAccelXDelta101,ImuAccelXDelta111,ImuAccelXDelta121,ImuAccelXDelta131,ImuAccelXDelta141,ImuAccelXDelta151,ImuAccelXDelta161,ImuAccelXDelta171,ImuAccelXDelta181,ImuAccelXDelta191,ImuAccelXDelta201,ImuAccelYDelta1,ImuAccelYDelta11,ImuAccelYDelta21,ImuAccelYDelta31,ImuAccelYDelta41,ImuAccelYDelta51,ImuAccelYDelta61,ImuAccelYDelta71,ImuAccelYDelta81,ImuAccelYDelta91,ImuAccelYDelta101,ImuAccelYDelta111,ImuAccelYDelta121,ImuAccelYDelta131,ImuAccelYDelta141,ImuAccelYDelta151,ImuAccelYDelta161,ImuAccelYDelta171,ImuAccelYDelta181,ImuAccelYDelta191,ImuAccelYDelta201,ImuAccelZDelta1,ImuAccelZDelta11,ImuAccelZDelta21,ImuAccelZDelta31,ImuAccelZDelta41,ImuAccelZDelta51,ImuAccelZDelta61,ImuAccelZDelta71,ImuAccelZDelta81,ImuAccelZDelta91,ImuAccelZDelta101,ImuAccelZDelta111,ImuAccelZDelta121,ImuAccelZDelta131,ImuAccelZDelta141,ImuAccelZDelta151,ImuAccelZDelta161,ImuAccelZDelta171,ImuAccelZDelta181,ImuAccelZDelta191,ImuAccelZDelta201,Speed,Terrain,Trial
Seq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,101.160756,7.681467,0.270914,0.962604,0.150111,-0.024509,0.006214,-0.012043,-0.546061,0.837631,-0.025541,0.004257,-0.043632,0.102951,0.313640,9.845555,0.005429,0.054662,0.128380,0.188695,0.249968,0.308652,0.371942,0.427930,0.490131,0.543923,0.605594,0.668520,0.723791,0.784910,0.846745,0.911929,0.966152,1.014465,1.072333,1.152219,1.211828,0.000617,0.009277,0.016039,0.024124,0.031259,0.039308,0.048162,0.054318,0.062553,0.068592,0.077931,0.084483,0.092145,0.099321,0.107379,0.116441,0.124210,0.132371,0.138544,0.148492,0.155278,0.000377,0.005678,...,-0.304662,-0.043694,-0.090381,-0.344765,0.323217,-0.441730,0.558448,-0.732626,-0.211288,0.536301,-0.307655,0.093374,-0.077812,-0.143054,-0.587477,0.000000,0.096965,0.215478,0.164003,-0.053870,0.058059,-0.122104,0.228048,0.086191,-0.092775,0.229843,-0.117914,0.228048,0.005387,0.098761,-0.079009,0.241814,0.265158,-0.004190,0.043096,-0.001197,0.000000,-0.536302,-0.465672,-0.232238,-0.632070,-0.566229,-0.295086,-0.994193,-0.975039,-0.657807,-0.286108,0.202310,0.351947,-0.878673,-1.385046,-1.148619,-0.013767,-0.643442,-0.423774,-0.792481,-0.213982,15,Turf,2
1,101.161373,7.681844,0.270831,0.962627,0.150933,-0.027563,0.005539,-0.011612,-0.546219,0.837539,-0.024477,0.000000,-0.052146,0.134075,0.226252,10.001179,0.004184,0.058805,0.124906,0.186766,0.239686,0.305206,0.367690,0.426726,0.483645,0.543607,0.605508,0.663000,0.723373,0.779015,0.840791,0.904299,0.965899,1.018624,1.076491,1.151998,1.208305,0.000617,0.009170,0.015659,0.024121,0.030322,0.039318,0.048160,0.054325,0.062564,0.068608,0.077938,0.084171,0.092159,0.098377,0.107379,0.116428,0.123829,0.132440,0.138439,0.148287,0.155278,0.000377,0.005612,...,-0.327108,0.109535,-0.231938,-0.520739,0.354342,-0.413598,0.589572,-0.322319,-0.174178,0.345663,-0.276530,0.219069,-0.046687,-0.207697,-0.432153,-0.087388,0.009577,0.145747,0.076614,0.003891,0.033220,-0.156820,0.243011,-0.079906,-0.183755,0.142455,-0.062549,0.140659,-0.087688,0.035614,-0.196325,0.154426,0.040701,-0.091578,-0.070629,-0.092177,0.155623,-0.380678,-0.146046,-0.07

In [207]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(248105, 376)

### Feature engineering: elimination

In [208]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Odom' in x) ]
df = df.drop(columns=columnsToDrop)

In [209]:
df.head(8)

,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXDelta1,OdomPosXDelta11,OdomPosXDelta21,OdomPosXDelta31,OdomPosXDelta41,OdomPosXDelta51,OdomPosXDelta61,OdomPosXDelta71,OdomPosXDelta81,OdomPosXDelta91,OdomPosXDelta101,OdomPosXDelta111,OdomPosXDelta121,OdomPosXDelta131,OdomPosXDelta141,OdomPosXDelta151,OdomPosXDelta161,OdomPosXDelta171,OdomPosXDelta181,OdomPosXDelta191,OdomPosXDelta201,OdomPosYDelta1,OdomPosYDelta11,OdomPosYDelta21,OdomPosYDelta31,OdomPosYDelta41,OdomPosYDelta51,OdomPosYDelta61,OdomPosYDelta71,OdomPosYDelta81,OdomPosYDelta91,OdomPosYDelta101,OdomPosYDelta111,OdomPosYDelta121,OdomPosYDelta131,OdomPosYDelta141,OdomPosYDelta151,OdomPosYDelta161,OdomPosYDelta171,OdomPosYDelta181,OdomPosYDelta191,OdomPosYDelta201,OdomOrientZDelta1,OdomOrientZDelta11,OdomOrientZDelta21,OdomOrientZDelta31,OdomOrientZDelta41,OdomOrientZDelta51,OdomOrientZDelta61,OdomOrientZDelta71,...,ImuAccelXDelta61,ImuAccelXDelta71,ImuAccelXDelta81,ImuAccelXDelta91,ImuAccelXDelta101,ImuAccelXDelta111,ImuAccelXDelta121,ImuAccelXDelta131,ImuAccelXDelta141,ImuAccelXDelta151,ImuAccelXDelta161,ImuAccelXDelta171,ImuAccelXDelta181,ImuAccelXDelta191,ImuAccelXDelta201,ImuAccelYDelta1,ImuAccelYDelta11,ImuAccelYDelta21,ImuAccelYDelta31,ImuAccelYDelta41,ImuAccelYDelta51,ImuAccelYDelta61,ImuAccelYDelta71,ImuAccelYDelta81,ImuAccelYDelta91,ImuAccelYDelta101,ImuAccelYDelta111,ImuAccelYDelta121,ImuAccelYDelta131,ImuAccelYDelta141,ImuAccelYDelta151,ImuAccelYDelta161,ImuAccelYDelta171,ImuAccelYDelta181,ImuAccelYDelta191,ImuAccelYDelta201,ImuAccelZDelta1,ImuAccelZDelta11,ImuAccelZDelta21,ImuAccelZDelta31,ImuAccelZDelta41,ImuAccelZDelta51,ImuAccelZDelta61,ImuAccelZDelta71,ImuAccelZDelta81,ImuAccelZDelta91,ImuAccelZDelta101,ImuAccelZDelta111,ImuAccelZDelta121,ImuAccelZDelta131,ImuAccelZDelta141,ImuAccelZDelta151,ImuAccelZDelta161,ImuAccelZDelta171,ImuAccelZDelta181,ImuAccelZDelta191,ImuAccelZDelta201,Speed,Terrain,Trial
Seq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.006214,-0.012043,-0.546061,0.837631,-0.025541,0.004257,-0.043632,0.102951,0.313640,9.845555,0.000617,0.009277,0.016039,0.024124,0.031259,0.039308,0.048162,0.054318,0.062553,0.068592,0.077931,0.084483,0.092145,0.099321,0.107379,0.116441,0.124210,0.132371,0.138544,0.148492,0.155278,0.000377,0.005678,0.009812,0.014753,0.019119,0.024057,0.029495,0.033272,0.038334,0.042054,0.047792,0.051818,0.056553,0.061005,0.066018,0.071654,0.076488,0.081589,0.085455,0.091710,0.096000,-0.000083,-0.000293,0.000326,-0.000124,-0.000437,-0.000939,-0.000821,-0.000759,...,-0.304662,-0.043694,-0.090381,-0.344765,0.323217,-0.441730,0.558448,-0.732626,-0.211288,0.536301,-0.307655,0.093374,-0.077812,-0.143054,-0.587477,0.000000,0.096965,0.215478,0.164003,-0.053870,0.058059,-0.122104,0.228048,0.086191,-0.092775,0.229843,-0.117914,0.228048,0.005387,0.098761,-0.079009,0.241814,0.265158,-0.004190,0.043096,-0.001197,0.000000,-0.536302,-0.465672,-0.232238,-0.632070,-0.566229,-0.295086,-0.994193,-0.975039,-0.657807,-0.286108,0.202310,0.351947,-0.878673,-1.385046,-1.148619,-0.013767,-0.643442,-0.423774,-0.792481,-0.213982,15,Turf,2
1,0.005539,-0.011612,-0.546219,0.837539,-0.024477,0.000000,-0.052146,0.134075,0.226252,10.001179,0.000617,0.009170,0.015659,0.024121,0.030322,0.039318,0.048160,0.054325,0.062564,0.068608,0.077938,0.084171,0.092159,0.098377,0.107379,0.116428,0.123829,0.132440,0.138439,0.148287,0.155278,0.000377,0.005612,0.009580,0.014751,0.018544,0.024061,0.029492,0.033275,0.038337,0.042062,0.047795,0.051621,0.056557,0.060411,0.066011,0.071639,0.076244,0.081623,0.085380,0.091568,0.095987,-0.000083,-0.000432,0.000307,-0.000182,-0.000358,-0.001008,-0.000919,-0.000836,...,-0.327108,0.109535,-0.231938,-0.520739,0.354342,-0.413598,0.589572,-0.322319,-0.174178,0.345663,-0.276530,0.219069,-0.046687,-0.207697,-0.432153,-0.087388,0.009577,0.145747,0.076614,0.003891,0.033220,-0.156820,0.243011,-0.0799

### Speed filtering

In [210]:
df = df[df['Speed']==10]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [211]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']==2]
dfTest = df[df['Trial']==1]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [212]:
featureCount = 30
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)
# print(fit.scores_)
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

In [213]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [214]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(29754, 30)
(29754,)
(29606, 30)
(29606,)


In [215]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [216]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.25910288455042896
